In [35]:
import os
import pandas as pd
import shutil
from pydub import AudioSegment
from tqdm import tqdm

# Define paths
DATASET_PATH = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample'
OUTPUT_DIR = os.path.join(DATASET_PATH, 'Preprocessed')
CSV_PATH = os.path.join(OUTPUT_DIR, 'audio_metadata.csv')

def ensure_dir_exists(directory):
    """Ensure directory exists, create if it doesn't."""
    if not os.path.exists(directory):
        os.makedirs(directory)

def convert_to_wav(input_path, output_path):
    """Convert audio file to WAV format if it's not already in WAV."""
    # Ensure output directory exists
    ensure_dir_exists(os.path.dirname(output_path))
    
    # Check if already WAV format
    if input_path.lower().endswith('.wav'):
        # Just copy the file
        shutil.copy2(input_path, output_path)
        return output_path
    
    # For other formats, convert to WAV
    try:
        if input_path.lower().endswith('.mp3'):
            audio = AudioSegment.from_mp3(input_path)
        elif input_path.lower().endswith('.flac'):
            audio = AudioSegment.from_file(input_path, format="flac")
        elif input_path.lower().endswith('.ogg'):
            audio = AudioSegment.from_ogg(input_path)
        elif input_path.lower().endswith('.m4a'):
            audio = AudioSegment.from_file(input_path, format="m4a")
        else:
            # Try to load using format inference
            audio = AudioSegment.from_file(input_path)
            
        # Export as WAV
        audio.export(output_path, format="wav")
        return output_path
    except Exception as e:
        print(f"Error converting {input_path}: {e}")
        return None

def process_dataset():
    """Process the dataset, converting files and creating a metadata CSV."""
    # Create main output directory
    ensure_dir_exists(OUTPUT_DIR)
    
    # Create separate directories for bio and non-bio sounds
    bio_output_dir = os.path.join(OUTPUT_DIR, 'bio')
    nonbio_output_dir = os.path.join(OUTPUT_DIR, 'non-bio')
    ensure_dir_exists(bio_output_dir)
    ensure_dir_exists(nonbio_output_dir)
    
    # Find all audio files in the dataset
    audio_files = []
    for root, _, files in os.walk(DATASET_PATH):
        # Skip the Preprocessed directory if it exists
        if 'Preprocessed' in root:
            continue
        
        for file in files:
            # Only process audio files
            if file.lower().endswith(('.wav', '.mp3', '.flac', '.ogg', '.m4a')):
                audio_files.append(os.path.join(root, file))
    
    print(f"Found {len(audio_files)} audio files in {DATASET_PATH}")
    
    # Process each file and build metadata
    metadata = []
    successful = 0
    failed = 0
    
    for file_path in tqdm(audio_files, desc="Processing audio files"):
        # Determine if file is biological sound based on folder structure or naming
        # This is a simplified example - adjust based on your actual dataset organization
        is_biological = 'bio' in os.path.basename(os.path.dirname(file_path)).lower()
        
        # Set output path
        if is_biological:
            output_dir = bio_output_dir
        else:
            output_dir = nonbio_output_dir
            
        # Get filename and create output path
        filename = os.path.basename(file_path)
        output_path = os.path.join(output_dir, filename)
        
        # Make sure output path ends with .wav
        if not output_path.lower().endswith('.wav'):
            output_path = os.path.splitext(output_path)[0] + '.wav'
        
        # Convert to WAV if needed
        result = convert_to_wav(file_path, output_path)
        
        if result:
            successful += 1
            # Extract species/type from filename or folder structure
            # This is a simplified approach - adjust based on your naming conventions
            folder_name = os.path.basename(os.path.dirname(file_path))
            species_type = folder_name if folder_name else "Unknown"
            
            # Add file to metadata
            metadata.append({
                'filename': os.path.basename(output_path),
                'path': output_path,
                'original_path': file_path,
                'is_biological': is_biological,
                'species_type': species_type,
                'preprocessed': False,  # Will be set to True after preprocessing
                'features_extracted': False  # Will be set to True after feature extraction
            })
        else:
            failed += 1
    
    # Create and save metadata DataFrame
    if metadata:
        df = pd.DataFrame(metadata)
        df.to_csv(CSV_PATH, index=False)
        print(f"Created metadata CSV with {len(metadata)} entries at {CSV_PATH}")
        
        # Print summary
        print("\nMetadata Summary:")
        print(f"Total files: {len(df)}")
        print(f"Biological sounds: {len(df[df['is_biological'] == True])}")
        print(f"Non-biological sounds: {len(df[df['is_biological'] == False])}")
        print(f"Successfully processed: {successful}")
        print(f"Failed to process: {failed}")
    else:
        print("No metadata created - no files were successfully processed.")
    
    return CSV_PATH if metadata else None

if __name__ == "__main__":
    csv_path = process_dataset()
    print(f"Process completed. Metadata saved to: {csv_path}")

Found 252 audio files in /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample


Processing audio files:  80%|███████▉  | 201/252 [00:00<00:00, 1014.30it/s]

Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Hydrothermal_vent/Hydrothermal Vent.mp3: [Errno 2] No such file or directory: 'ffprobe'
Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Volcanos/undersea-volcano-eruptions.mp3: [Errno 2] No such file or directory: 'ffprobe'
Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Volcanos/underwater-volcanoes.mp3: [Errno 2] No such file or directory: 'ffprobe'
Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Submarine/Submarine3.mp3: [Errno 2] No such file or directory: 'ffprobe'
Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Submarine/Submarine2.mp3: [Errno 2] No such file or di

Processing audio files: 100%|██████████| 252/252 [00:00<00:00, 441.08it/s] 

Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Hurricane/hurricane6.mp3: [Errno 2] No such file or directory: 'ffprobe'
Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Hurricane/Hurricane.mp3: [Errno 2] No such file or directory: 'ffprobe'
Error converting /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio/Icrberg collisions/Iceberg-collisions.mp3: [Errno 2] No such file or directory: 'ffprobe'
Created metadata CSV with 213 entries at /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/audio_metadata.csv

Metadata Summary:
Total files: 213
Biological sounds: 0
Non-biological sounds: 213
Successfully processed: 213
Failed to process: 39
Process completed. Metadata saved to: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Desig

## preprocessing

In [36]:
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, resample_poly, stft, istft
from scipy.io import wavfile
from tqdm import tqdm
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# Define paths
DATASET_PATH = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample'
OUTPUT_DIR = os.path.join(DATASET_PATH, 'Preprocessed')
CSV_PATH = os.path.join(OUTPUT_DIR, 'audio_metadata.csv')

def butter_bandpass(lowcut, highcut, fs, order=2):
    """Design a Butterworth bandpass filter with safety checks."""
    nyquist = 0.5 * fs
    
    # Make sure cutoffs are within valid range (0 < Wn < 1)
    low = max(0.001, min(0.999, lowcut / nyquist))
    
    # Ensure highcut is below Nyquist
    safe_highcut = min(highcut, 0.95 * nyquist)
    high = max(0.001, min(0.999, safe_highcut / nyquist))
    
    # Extra check to ensure low < high
    if low >= high:
        print(f"WARNING: Invalid filter frequencies after normalization: low={low}, high={high}")
        print(f"Setting low to 0.01 and high to 0.9")
        low, high = 0.01, 0.9
    
    b, a = butter(order, [low, high], btype='band')
    return b, a

def apply_bandpass_filter(data, fs, lowcut=50, highcut=24000, order=2):
    """Apply bandpass filter to audio data with safer highcut."""
    try:
        b, a = butter_bandpass(lowcut, highcut, fs)
        filtered_data = filtfilt(b, a, data)
        return filtered_data
    except Exception as e:
        print(f"ERROR: Filter design failed: {e}")
        print("Using original unfiltered audio")
        return data  # Return original if filter fails

def resample_audio(audio, orig_sr, target_sr=48000):
    """Resample audio to target sampling rate."""
    try:
        # Calculate greatest common divisor for resampling
        gcd = np.gcd(orig_sr, target_sr)
        up = target_sr // gcd
        down = orig_sr // gcd
        
        # Resample using scipy
        resampled = resample_poly(audio, up, down)
        return resampled
    except Exception as e:
        print(f"ERROR: Resampling failed: {e}")
        # Pad or truncate to simulate resampling
        target_length = int(len(audio) * target_sr / orig_sr)
        if target_length > len(audio):
            return np.pad(audio, (0, target_length - len(audio)))
        else:
            return audio[:target_length]

def reduce_noise_spectral_subtraction(audio, sr):
    """Reduce noise using spectral subtraction."""
    try:
        # Parameters for STFT
        nperseg = 2048  # Window size
        noverlap = nperseg // 2  # 50% overlap
        
        # Compute STFT
        f, t, Zxx = stft(audio, fs=sr, nperseg=nperseg, noverlap=noverlap)
        
        # Get magnitude and phase
        magnitude = np.abs(Zxx)
        phase = np.angle(Zxx)
        
        # Estimate noise floor as minimum value per frequency bin
        noise_floor = np.min(magnitude, axis=1, keepdims=True)
        
        # Subtract noise floor from magnitude with a small buffer to avoid negative values
        magnitude_clean = np.maximum(magnitude - noise_floor, 0.01 * noise_floor)
        
        # Reconstruct complex STFT
        Zxx_clean = magnitude_clean * np.exp(1j * phase)
        
        # Inverse STFT
        _, audio_clean = istft(Zxx_clean, fs=sr, nperseg=nperseg, noverlap=noverlap)
        
        # Ensure same length as original
        if len(audio_clean) > len(audio):
            audio_clean = audio_clean[:len(audio)]
        elif len(audio_clean) < len(audio):
            audio_clean = np.pad(audio_clean, (0, len(audio) - len(audio_clean)))
        
        # Normalize to prevent clipping
        if np.max(np.abs(audio_clean)) > 0:
            audio_clean = audio_clean / np.max(np.abs(audio_clean))
        
        return audio_clean
    except Exception as e:
        print(f"ERROR: Noise reduction failed: {e}")
        return audio  # Return original if noise reduction fails

def preprocess_audio(file_path, output_path=None):
    """Apply preprocessing to an audio file."""
    try:
        # If output path is not specified, create one
        if output_path is None:
            dir_path = os.path.dirname(file_path)
            filename = os.path.basename(file_path)
            output_path = os.path.join(dir_path, f"preprocessed_{filename}")
        
        # Ensure directory exists
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # Load audio file
        try:
            sr, audio = wavfile.read(file_path)
            # Print sampling rate for debugging
            print(f"Processing file: {file_path}, sample rate: {sr} Hz")
            
            # Convert to float32 normalized to [-1, 1] for processing
            if audio.dtype == np.int16:
                audio = audio.astype(np.float32) / 32767.0
            elif audio.dtype == np.int32:
                audio = audio.astype(np.float32) / 2147483647.0
                
            # If stereo, convert to mono
            if len(audio.shape) > 1 and audio.shape[1] > 1:
                audio = np.mean(audio, axis=1)
        except Exception as e:
            print(f"ERROR: Failed to load audio file: {e}")
            return None
        
        # Step 1: Resample to 48 kHz if needed
        target_sr = 48000
        if sr != target_sr:
            try:
                audio = resample_audio(audio, sr, target_sr)
                sr = target_sr
                print(f"Resampled to {sr} Hz")
            except Exception as e:
                print(f"ERROR: Failed to resample: {e}")
                return None
        
        # Step 2: Apply bandpass filter (50 Hz - 24 kHz)
        # Adjust highcut based on sampling rate to avoid filter errors
        highcut = min(24000, sr/2 * 0.95)  # Ensure highcut is below Nyquist
        audio_filtered = apply_bandpass_filter(audio, sr, lowcut=50, highcut=highcut)
        print("Applied bandpass filter")
        
        # Step 3: Reduce noise
        audio_clean = reduce_noise_spectral_subtraction(audio_filtered, sr)
        print("Applied noise reduction")
        
        # Convert back to int16 for saving
        audio_int16 = np.int16(audio_clean * 32767)
        
        # Save the processed audio
        try:
            wavfile.write(output_path, sr, audio_int16)
            print(f"Saved preprocessed audio to {output_path}")
            return output_path
        except Exception as e:
            print(f"ERROR: Failed to save processed audio: {e}")
            return None
    
    except Exception as e:
        print(f"ERROR: Unexpected error preprocessing {file_path}: {e}")
        return None

def preprocess_from_csv(csv_path, max_files=None):
    """Preprocess all audio files listed in the CSV."""
    # Check if CSV exists
    if not os.path.exists(csv_path):
        print(f"CSV file not found: {csv_path}")
        return None
    
    # Load metadata
    try:
        df = pd.read_csv(csv_path)
        print(f"Loaded CSV with {len(df)} entries")
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None
    
    # Limit number of files to process if specified
    if max_files is not None:
        df = df.head(max_files)
        print(f"Limited to processing {max_files} files")
    
    # Track success and failure
    successful = 0
    failed = 0
    
    # Process each file
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Preprocessing audio"):
        # Skip already preprocessed files
        if row.get('preprocessed', False):
            continue
        
        file_path = row['path']
        
        # Check if file exists
        if not os.path.exists(file_path):
            print(f"File not found: {file_path}")
            failed += 1
            continue
        
        # Create output path with prefix
        dir_path = os.path.dirname(file_path)
        filename = os.path.basename(file_path)
        output_path = os.path.join(dir_path, f"preprocessed_{filename}")
        
        # Preprocess the audio
        result = preprocess_audio(file_path, output_path)
        
        if result:
            # Update dataframe
            df.loc[idx, 'preprocessed'] = True
            df.loc[idx, 'preprocessed_path'] = result
            successful += 1
            
            # Save progress periodically
            if (successful + failed) % 10 == 0:
                df.to_csv(csv_path, index=False)
        else:
            failed += 1
    
    # Final save
    df.to_csv(csv_path, index=False)
    
    print(f"Preprocessing complete. Success: {successful}, Failed: {failed}")
    return df

if __name__ == "__main__":
    df = preprocess_from_csv(CSV_PATH)
    if df is not None:
        # Print summary
        print("\nPreprocessing Summary:")
        print(f"Total files: {len(df)}")
        print(f"Successfully preprocessed: {len(df[df['preprocessed'] == True])}")
        print(f"Not preprocessed: {len(df[df['preprocessed'] == False])}")

Loaded CSV with 213 entries


Preprocessing audio:   4%|▍         | 8/213 [00:00<00:02, 70.13it/s]

Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6102101A.wav, sample rate: 81920 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6102101A.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6102101P.wav, sample rate: 81920 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6102101P.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6102104K.wav, sample rate: 81600 Hz
Resampled to 48000 Hz
Applied bandpass filt

Preprocessing audio:   8%|▊         | 16/213 [00:00<00:02, 74.80it/s]

Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6002600P.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6403000K.wav, sample rate: 40000 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6403000K.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6002604X.wav, sample rate: 45400 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6002604X.wav
Processing file: /

...

Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6400800Y.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6400900B.wav, sample rate: 60600 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6400900B.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6400900G.wav, sample rate: 60600 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6400900

Preprocessing audio:  97%|█████████▋| 207/213 [00:06<00:00, 49.89it/s]

Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_66002A01.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/6609000L.wav, sample rate: 20000 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6609000L.wav
Processing file: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/66002A02.wav, sample rate: 30000 Hz
Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_66002A02.wav
Processing file: /

Preprocessing audio: 100%|██████████| 213/213 [00:06<00:00, 32.50it/s]

Resampled to 48000 Hz
Applied bandpass filter
Applied noise reduction
Saved preprocessed audio to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_Earthquake sound3.wav
Preprocessing complete. Success: 213, Failed: 0

Preprocessing Summary:
Total files: 213
Successfully preprocessed: 213
Not preprocessed: 0


## 3. feature Extraction

In [37]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from tqdm import tqdm
import warnings
import torch
import time
import signal
from contextlib import contextmanager
import threading

# Suppress warnings
warnings.filterwarnings('ignore')

# Define paths
DATASET_PATH = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample'
OUTPUT_DIR = os.path.join(DATASET_PATH, 'Preprocessed')
CSV_PATH = os.path.join(OUTPUT_DIR, 'audio_metadata.csv')
FEATURES_DIR = os.path.join(OUTPUT_DIR, 'Features')

# Configuration
BATCH_SIZE = 10              # Process files in batches (smaller batch size)
FILE_TIMEOUT = 30            # Maximum seconds to spend on any single file
VERBOSE = True               # Print detailed information for debugging

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Metal) device for acceleration")
else:
    device = torch.device("cpu")
    print("MPS not available, using CPU instead")

# Timeout handler for individual file processing
class TimeoutError(Exception):
    pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutError("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

# Alternative timeout mechanism using threading
def process_with_timeout(func, args, timeout):
    result = [None]
    exception = [None]
    
    def worker():
        try:
            result[0] = func(*args)
        except Exception as e:
            exception[0] = e
    
    thread = threading.Thread(target=worker)
    thread.daemon = True
    thread.start()
    thread.join(timeout)
    
    if thread.is_alive():
        return None, TimeoutError(f"Processing timed out after {timeout} seconds")
    
    if exception[0] is not None:
        return None, exception[0]
    
    return result[0], None

def ensure_dirs_exist():
    """Create all necessary directories."""
    dirs = [
        FEATURES_DIR,
        os.path.join(FEATURES_DIR, 'Spectrograms'),
        os.path.join(FEATURES_DIR, 'STFT'),
        os.path.join(FEATURES_DIR, 'MFCC_Plots'),
        os.path.join(FEATURES_DIR, 'MFCC')
    ]
    for d in dirs:
        os.makedirs(d, exist_ok=True)

def extract_stft_features(audio_tensor, sr, file_id, use_gpu=True):
    """Extract STFT features using PyTorch with fallback to CPU if GPU fails."""
    try:
        # Create output paths
        stft_vis_path = os.path.join(FEATURES_DIR, 'Spectrograms', f"{file_id}_spectrogram.png")
        stft_data_path = os.path.join(FEATURES_DIR, 'STFT', f"{file_id}_stft.npy")
        
        # Compute STFT using PyTorch
        n_fft = min(2048, audio_tensor.shape[0]//2)
        hop_length = n_fft // 4
        
        # Try GPU processing first if requested
        if use_gpu:
            try:
                window = torch.hann_window(n_fft).to(device)
                audio_gpu = audio_tensor.to(device)
                
                stft_complex = torch.stft(
                    audio_gpu, 
                    n_fft=n_fft, 
                    hop_length=hop_length,
                    window=window,
                    return_complex=True
                )
                
                # Convert to magnitude spectrogram in dB scale
                S_mag = torch.abs(stft_complex)
                S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
                
                # Move back to CPU for visualization
                S_db_np = S_db.cpu().numpy()
            except Exception as e:
                print(f"GPU processing failed for STFT: {e}. Falling back to CPU.")
                use_gpu = False
        
        # Fall back to CPU if GPU failed or wasn't requested
        if not use_gpu:
            window = torch.hann_window(n_fft)
            stft_complex = torch.stft(
                audio_tensor, 
                n_fft=n_fft, 
                hop_length=hop_length,
                window=window,
                return_complex=True
            )
            
            # Convert to magnitude spectrogram in dB scale
            S_mag = torch.abs(stft_complex)
            S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
            S_db_np = S_db.numpy()
        
        # Create spectrogram visualization
        plt.figure(figsize=(10, 6))
        plt.pcolormesh(np.arange(S_db_np.shape[1]), np.arange(S_db_np.shape[0]), S_db_np, shading='gouraud')
        plt.ylabel('Frequency Bin')
        plt.xlabel('Time Frame')
        plt.title(f'Spectrogram - {file_id}')
        plt.colorbar(label='Magnitude [dB]')
        plt.tight_layout()
        
        # Save the spectrogram visualization
        plt.savefig(stft_vis_path)
        plt.close()
        
        # Save the STFT data
        np.save(stft_data_path, S_db_np)
        
        return S_db_np, stft_vis_path, stft_data_path
    except Exception as e:
        print(f"Error extracting STFT features for {file_id}: {e}")
        return None, None, None

def extract_mfcc_like_features(audio_tensor, sr, file_id, use_gpu=True):
    """Create a simplified MFCC-like feature with fallback to CPU if GPU fails."""
    try:
        # Create output paths
        mfcc_vis_path = os.path.join(FEATURES_DIR, 'MFCC_Plots', f"{file_id}_mfcc.png")
        mfcc_data_path = os.path.join(FEATURES_DIR, 'MFCC', f"{file_id}_mfcc.npy")
        
        # Just grab the STFT with different parameters for MFCC
        n_fft = min(2048, audio_tensor.shape[0]//2)
        hop_length = n_fft // 4
        
        # Try GPU processing first if requested
        if use_gpu:
            try:
                window = torch.hann_window(n_fft).to(device)
                audio_gpu = audio_tensor.to(device)
                
                stft_complex = torch.stft(
                    audio_gpu, 
                    n_fft=n_fft, 
                    hop_length=hop_length,
                    window=window,
                    return_complex=True
                )
                
                # Convert to magnitude
                S_mag = torch.abs(stft_complex)
                S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
                
                # Take the lower part of the spectrogram for MFCC-like features
                n_mfcc = min(40, S_db.shape[0]//2)
                mfcc_like = S_db[:n_mfcc]
                
                # Move back to CPU for visualization
                mfcc_like_np = mfcc_like.cpu().numpy()
            except Exception as e:
                print(f"GPU processing failed for MFCC: {e}. Falling back to CPU.")
                use_gpu = False
        
        # Fall back to CPU if GPU failed or wasn't requested
        if not use_gpu:
            window = torch.hann_window(n_fft)
            stft_complex = torch.stft(
                audio_tensor, 
                n_fft=n_fft, 
                hop_length=hop_length,
                window=window,
                return_complex=True
            )
            
            # Convert to magnitude
            S_mag = torch.abs(stft_complex)
            S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
            
            # Take the lower part of the spectrogram for MFCC-like features
            n_mfcc = min(40, S_db.shape[0]//2)
            mfcc_like = S_db[:n_mfcc]
            mfcc_like_np = mfcc_like.numpy()
        
        # Create visualization
        plt.figure(figsize=(10, 6))
        plt.pcolormesh(np.arange(mfcc_like_np.shape[1]), np.arange(mfcc_like_np.shape[0]), 
                      mfcc_like_np, shading='gouraud')
        plt.ylabel('MFCC Coefficient')
        plt.xlabel('Time Frame')
        plt.title(f'MFCC-like Features - {file_id}')
        plt.colorbar(label='Magnitude [dB]')
        plt.tight_layout()
        
        # Save the visualization
        plt.savefig(mfcc_vis_path)
        plt.close()
        
        # Save the data
        np.save(mfcc_data_path, mfcc_like_np)
        
        return mfcc_like_np, mfcc_vis_path, mfcc_data_path
    except Exception as e:
        print(f"Error extracting MFCC features for {file_id}: {e}")
        return None, None, None

def process_file(file_path):
    """Process a single file and extract features with timeout."""
    file_id = os.path.splitext(os.path.basename(file_path))[0]
    print(f"Starting to process: {file_id}")
    
    try:
        # Check file size first and warn if it's too large
        file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
        if file_size_mb > 100:
            print(f"WARNING: File {file_id} is large: {file_size_mb:.2f} MB")
        
        # Load audio
        try:
            sr, audio = wavfile.read(file_path)
            
            # Log file details for debugging
            print(f"Loaded file: {file_id}, Sample rate: {sr}, Shape: {audio.shape}, Size: {file_size_mb:.2f} MB")
            
            # Convert to float32 normalized to [-1, 1] for processing
            if audio.dtype == np.int16:
                audio = audio.astype(np.float32) / 32767.0
            elif audio.dtype == np.int32:
                audio = audio.astype(np.float32) / 2147483647.0
                
            # If stereo, convert to mono
            if len(audio.shape) > 1 and audio.shape[1] > 1:
                audio = np.mean(audio, axis=1)
            
            # Convert to PyTorch tensor
            audio_tensor = torch.tensor(audio, dtype=torch.float32)
            
        except Exception as e:
            print(f"ERROR: Failed to load audio file {file_path}: {e}")
            return None
        
        # Use CPU mode for larger files
        use_gpu = file_size_mb < 50  # Use CPU for files larger than 50MB
        device_str = "GPU" if use_gpu else "CPU"
        print(f"Processing {file_id} using {device_str}")
        
        # Extract STFT features with timeout
        stft_result, stft_error = process_with_timeout(
            extract_stft_features, 
            (audio_tensor, sr, file_id, use_gpu), 
            FILE_TIMEOUT//2
        )
        
        if stft_error:
            print(f"STFT extraction timed out or failed for {file_id}: {stft_error}")
            stft_features, stft_vis_path, stft_data_path = None, None, None
        else:
            stft_features, stft_vis_path, stft_data_path = stft_result
        
        # Extract MFCC-like features with timeout
        mfcc_result, mfcc_error = process_with_timeout(
            extract_mfcc_like_features, 
            (audio_tensor, sr, file_id, use_gpu), 
            FILE_TIMEOUT//2
        )
        
        if mfcc_error:
            print(f"MFCC extraction timed out or failed for {file_id}: {mfcc_error}")
            mfcc_features, mfcc_vis_path, mfcc_data_path = None, None, None
        else:
            mfcc_features, mfcc_vis_path, mfcc_data_path = mfcc_result
        
        # Check if at least one extraction was successful
        if stft_data_path or mfcc_data_path:
            feature_info = {
                'stft_path': stft_data_path,
                'stft_vis_path': stft_vis_path,
                'mfcc_path': mfcc_data_path,
                'mfcc_vis_path': mfcc_vis_path
            }
            print(f"Successfully processed {file_id}")
            return feature_info
        else:
            print(f"Feature extraction failed for {file_id}")
            return None
            
    except TimeoutError:
        print(f"ERROR: Processing timed out for {file_id}")
        return None
    except Exception as e:
        print(f"ERROR: Unexpected error processing {file_id}: {e}")
        return None

def extract_features_from_csv():
    """Extract features with improved error handling and timeouts."""
    # Ensure directories exist
    ensure_dirs_exist()
    
    # Check if CSV exists
    if not os.path.exists(CSV_PATH):
        print(f"ERROR: CSV file not found: {CSV_PATH}")
        return None
    
    # Load metadata
    try:
        df = pd.read_csv(CSV_PATH)
        print(f"Loaded CSV with {len(df)} entries")
    except Exception as e:
        print(f"ERROR: Failed to load CSV: {e}")
        return None
    
    # Add feature columns if they don't exist
    for col in ['stft_path', 'stft_vis_path', 'mfcc_path', 'mfcc_vis_path', 'features_extracted']:
        if col not in df.columns:
            df[col] = None if col != 'features_extracted' else False
    
    # Find files that need processing - look for preprocessed files
    if 'preprocessed' in df.columns and 'preprocessed_path' in df.columns:
        # Use preprocessed files if available
        to_process = df[(df['preprocessed'] == True) & 
                         (df['features_extracted'] != True) & 
                         (df['preprocessed_path'].notna())]
        file_path_col = 'preprocessed_path'
        print(f"Found {len(to_process)} preprocessed files that need feature extraction")
    else:
        # Otherwise use original files
        to_process = df[df['features_extracted'] != True]
        file_path_col = 'path'
        print(f"No preprocessed files found, using {len(to_process)} original files")
    
    if len(to_process) == 0:
        print("No files to process! Either they're all processed or none are preprocessed.")
        return df
    
    # Track success and failure
    successful = 0
    failed = 0
    
    # Process files individually - batching changed to sequential to better identify issues
    for i, (idx, row) in enumerate(tqdm(to_process.iterrows(), total=len(to_process), desc="Processing files")):
        print(f"\nProcessing file {i+1}/{len(to_process)}")
        
        file_path = row[file_path_col]
        
        # Check if file exists
        if not isinstance(file_path, str) or not os.path.exists(file_path):
            print(f"File not found or invalid path: {file_path}")
            failed += 1
            continue
        
        # Process the file with overall timeout
        try:
            # Use the timeout mechanism
            start_time = time.time()
            feature_info = process_file(file_path)
            end_time = time.time()
            
            if feature_info:
                # Update dataframe with feature paths
                for key, value in feature_info.items():
                    df.loc[idx, key] = value
                
                df.loc[idx, 'features_extracted'] = True
                successful += 1
                print(f"File processed successfully in {end_time - start_time:.2f} seconds")
            else:
                failed += 1
                print(f"File processing failed after {end_time - start_time:.2f} seconds")
            
            # Save progress after each file
            if (i + 1) % 5 == 0:
                df.to_csv(CSV_PATH, index=False)
                print(f"Progress saved. Processed {i+1}/{len(to_process)} files")
                
        except Exception as e:
            print(f"ERROR: Failed to process file {file_path}: {e}")
            failed += 1
    
    # Final save
    df.to_csv(CSV_PATH, index=False)
    
    print(f"\nFeature extraction complete. Success: {successful}, Failed: {failed}")
    return df

# Run the extraction
if __name__ == "__main__":
    print(f"PyTorch version: {torch.__version__}")
    print(f"MPS available: {torch.backends.mps.is_available()}")
    print(f"MPS built: {torch.backends.mps.is_built()}")
    
    start_time = time.time()
    print(f"Starting feature extraction from {CSV_PATH}")
    
    df = extract_features_from_csv()
    
    if df is not None:
        end_time = time.time()
        print(f"\nFeature extraction completed in {end_time - start_time:.2f} seconds!")
        
        # Create visualizations if we have any extracted features
        if len(df[df['features_extracted'] == True]) > 0:
            from create_visualizations import create_feature_visualizations
            create_feature_visualizations(df)
            print("Feature visualizations created")
    else:
        print("\nFeature extraction failed. Check the errors above.")

Using MPS (Apple Metal) device for acceleration
PyTorch version: 2.6.0
MPS available: True
MPS built: True
Starting feature extraction from /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/audio_metadata.csv
Loaded CSV with 213 entries
Found 213 preprocessed files that need feature extraction


Processing files:   0%|          | 0/213 [00:00<?, ?it/s]


Processing file 1/213
Starting to process: preprocessed_6102101A
Loaded file: preprocessed_6102101A, Sample rate: 48000, Shape: (23175,), Size: 0.04 MB
Processing preprocessed_6102101A using GPU


Processing files:   0%|          | 1/213 [00:01<06:31,  1.85s/it]

Successfully processed preprocessed_6102101A
File processed successfully in 1.85 seconds

Processing file 2/213
Starting to process: preprocessed_6102101P
Loaded file: preprocessed_6102101P, Sample rate: 48000, Shape: (34425,), Size: 0.07 MB
Processing preprocessed_6102101P using GPU


...

Processing files: 100%|█████████▉| 212/213 [16:51<00:02,  2.20s/it]

Successfully processed preprocessed_ice-cracking
File processed successfully in 2.20 seconds

Processing file 213/213
Starting to process: preprocessed_Earthquake sound3
Loaded file: preprocessed_Earthquake sound3, Sample rate: 48000, Shape: (1447937,), Size: 2.76 MB
Processing preprocessed_Earthquake sound3 using GPU


Processing files: 100%|██████████| 213/213 [17:05<00:00,  4.81s/it]

Successfully processed preprocessed_Earthquake sound3
File processed successfully in 13.89 seconds

Feature extraction complete. Success: 196, Failed: 17

Feature extraction completed in 1025.35 seconds!


ModuleNotFoundError: No module named 'create_visualizations'

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from tqdm import tqdm
import warnings
import torch
import time
import signal
from contextlib import contextmanager
import threading

# Suppress warnings
warnings.filterwarnings('ignore')

# Define paths
DATASET_PATH = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample'
OUTPUT_DIR = os.path.join(DATASET_PATH, 'Preprocessed')
CSV_PATH = os.path.join(OUTPUT_DIR, 'audio_metadata.csv')
FEATURES_DIR = os.path.join(OUTPUT_DIR, 'Features')

# Configuration
BATCH_SIZE = 10              # Process files in batches
FILE_TIMEOUT = 30            # Maximum seconds to spend on any single file
VERBOSE = True               # Print detailed information for debugging

# Check if MPS is available
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Apple Metal) device for acceleration")
else:
    device = torch.device("cpu")
    print("MPS not available, using CPU instead")

# Timeout handler for individual file processing
class TimeoutError(Exception):
    pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutError("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

# Alternative timeout mechanism using threading
def process_with_timeout(func, args, timeout):
    result = [None]
    exception = [None]
    
    def worker():
        try:
            result[0] = func(*args)
        except Exception as e:
            exception[0] = e
    
    thread = threading.Thread(target=worker)
    thread.daemon = True
    thread.start()
    thread.join(timeout)
    
    if thread.is_alive():
        return None, TimeoutError(f"Processing timed out after {timeout} seconds")
    
    if exception[0] is not None:
        return None, exception[0]
    
    return result[0], None

def ensure_dirs_exist():
    """Create all necessary directories."""
    dirs = [
        FEATURES_DIR,
        os.path.join(FEATURES_DIR, 'Spectrograms'),
        os.path.join(FEATURES_DIR, 'STFT'),
        os.path.join(FEATURES_DIR, 'MFCC_Plots'),
        os.path.join(FEATURES_DIR, 'MFCC')
    ]
    for d in dirs:
        os.makedirs(d, exist_ok=True)

def extract_stft_features(audio_tensor, sr, file_id, use_gpu=True):
    """Extract STFT features using PyTorch with fallback to CPU if GPU fails."""
    try:
        # Create output paths
        stft_vis_path = os.path.join(FEATURES_DIR, 'Spectrograms', f"{file_id}_spectrogram.png")
        stft_data_path = os.path.join(FEATURES_DIR, 'STFT', f"{file_id}_stft.npy")
        
        # Compute STFT using PyTorch
        n_fft = min(2048, audio_tensor.shape[0]//2)
        hop_length = n_fft // 4
        
        # Try GPU processing first if requested
        if use_gpu:
            try:
                window = torch.hann_window(n_fft).to(device)
                audio_gpu = audio_tensor.to(device)
                
                stft_complex = torch.stft(
                    audio_gpu, 
                    n_fft=n_fft, 
                    hop_length=hop_length,
                    window=window,
                    return_complex=True
                )
                
                # Convert to magnitude spectrogram in dB scale
                S_mag = torch.abs(stft_complex)
                S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
                
                # Move back to CPU for visualization
                S_db_np = S_db.cpu().numpy()
            except Exception as e:
                print(f"GPU processing failed for STFT: {e}. Falling back to CPU.")
                use_gpu = False
        
        # Fall back to CPU if GPU failed or wasn't requested
        if not use_gpu:
            window = torch.hann_window(n_fft)
            stft_complex = torch.stft(
                audio_tensor, 
                n_fft=n_fft, 
                hop_length=hop_length,
                window=window,
                return_complex=True
            )
            
            # Convert to magnitude spectrogram in dB scale
            S_mag = torch.abs(stft_complex)
            S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
            S_db_np = S_db.numpy()
        
        # Create spectrogram visualization
        plt.figure(figsize=(10, 6))
        plt.pcolormesh(np.arange(S_db_np.shape[1]), np.arange(S_db_np.shape[0]), S_db_np, shading='gouraud')
        plt.ylabel('Frequency Bin')
        plt.xlabel('Time Frame')
        plt.title(f'Spectrogram - {file_id}')
        plt.colorbar(label='Magnitude [dB]')
        plt.tight_layout()
        
        # Save the spectrogram visualization
        plt.savefig(stft_vis_path)
        plt.close()
        
        # Save the STFT data
        np.save(stft_data_path, S_db_np)
        
        return S_db_np, stft_vis_path, stft_data_path
    except Exception as e:
        print(f"Error extracting STFT features for {file_id}: {e}")
        return None, None, None

def extract_mfcc_like_features(audio_tensor, sr, file_id, use_gpu=True):
    """Create a simplified MFCC-like feature with fallback to CPU if GPU fails."""
    try:
        # Create output paths
        mfcc_vis_path = os.path.join(FEATURES_DIR, 'MFCC_Plots', f"{file_id}_mfcc.png")
        mfcc_data_path = os.path.join(FEATURES_DIR, 'MFCC', f"{file_id}_mfcc.npy")
        
        # Just grab the STFT with different parameters for MFCC
        n_fft = min(2048, audio_tensor.shape[0]//2)
        hop_length = n_fft // 4
        
        # Try GPU processing first if requested
        if use_gpu:
            try:
                window = torch.hann_window(n_fft).to(device)
                audio_gpu = audio_tensor.to(device)
                
                stft_complex = torch.stft(
                    audio_gpu, 
                    n_fft=n_fft, 
                    hop_length=hop_length,
                    window=window,
                    return_complex=True
                )
                
                # Convert to magnitude
                S_mag = torch.abs(stft_complex)
                S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
                
                # Take the lower part of the spectrogram for MFCC-like features
                n_mfcc = min(40, S_db.shape[0]//2)
                mfcc_like = S_db[:n_mfcc]
                
                # Move back to CPU for visualization
                mfcc_like_np = mfcc_like.cpu().numpy()
            except Exception as e:
                print(f"GPU processing failed for MFCC: {e}. Falling back to CPU.")
                use_gpu = False
        
        # Fall back to CPU if GPU failed or wasn't requested
        if not use_gpu:
            window = torch.hann_window(n_fft)
            stft_complex = torch.stft(
                audio_tensor, 
                n_fft=n_fft, 
                hop_length=hop_length,
                window=window,
                return_complex=True
            )
            
            # Convert to magnitude
            S_mag = torch.abs(stft_complex)
            S_db = 20 * torch.log10(torch.clamp(S_mag, min=1e-10))
            
            # Take the lower part of the spectrogram for MFCC-like features
            n_mfcc = min(40, S_db.shape[0]//2)
            mfcc_like = S_db[:n_mfcc]
            mfcc_like_np = mfcc_like.numpy()
        
        # Create visualization
        plt.figure(figsize=(10, 6))
        plt.pcolormesh(np.arange(mfcc_like_np.shape[1]), np.arange(mfcc_like_np.shape[0]), 
                      mfcc_like_np, shading='gouraud')
        plt.ylabel('MFCC Coefficient')
        plt.xlabel('Time Frame')
        plt.title(f'MFCC-like Features - {file_id}')
        plt.colorbar(label='Magnitude [dB]')
        plt.tight_layout()
        
        # Save the visualization
        plt.savefig(mfcc_vis_path)
        plt.close()
        
        # Save the data
        np.save(mfcc_data_path, mfcc_like_np)
        
        return mfcc_like_np, mfcc_vis_path, mfcc_data_path
    except Exception as e:
        print(f"Error extracting MFCC features for {file_id}: {e}")
        return None, None, None

def process_file(file_path):
    """Process a single file and extract features with timeout."""
    file_id = os.path.splitext(os.path.basename(file_path))[0]
    print(f"Starting to process: {file_id}")
    
    try:
        # Check file size first and warn if it's too large
        file_size_mb = os.path.getsize(file_path) / (1024 * 1024)
        if file_size_mb > 100:
            print(f"WARNING: File {file_id} is large: {file_size_mb:.2f} MB")
        
        # Load audio
        try:
            sr, audio = wavfile.read(file_path)
            
            # Log file details for debugging
            print(f"Loaded file: {file_id}, Sample rate: {sr}, Shape: {audio.shape}, Size: {file_size_mb:.2f} MB")
            
            # Convert to float32 normalized to [-1, 1] for processing
            if audio.dtype == np.int16:
                audio = audio.astype(np.float32) / 32767.0
            elif audio.dtype == np.int32:
                audio = audio.astype(np.float32) / 2147483647.0
                
            # If stereo, convert to mono
            if len(audio.shape) > 1 and audio.shape[1] > 1:
                audio = np.mean(audio, axis=1)
            
            # Convert to PyTorch tensor
            audio_tensor = torch.tensor(audio, dtype=torch.float32)
            
        except Exception as e:
            print(f"ERROR: Failed to load audio file {file_path}: {e}")
            return None
        
        # Use CPU mode for larger files
        use_gpu = file_size_mb < 50  # Use CPU for files larger than 50MB
        device_str = "GPU" if use_gpu else "CPU"
        print(f"Processing {file_id} using {device_str}")
        
        # Extract STFT features with timeout
        stft_result, stft_error = process_with_timeout(
            extract_stft_features, 
            (audio_tensor, sr, file_id, use_gpu), 
            FILE_TIMEOUT//2
        )
        
        if stft_error:
            print(f"STFT extraction timed out or failed for {file_id}: {stft_error}")
            stft_features, stft_vis_path, stft_data_path = None, None, None
        else:
            stft_features, stft_vis_path, stft_data_path = stft_result
        
        # Extract MFCC-like features with timeout
        mfcc_result, mfcc_error = process_with_timeout(
            extract_mfcc_like_features, 
            (audio_tensor, sr, file_id, use_gpu), 
            FILE_TIMEOUT//2
        )
        
        if mfcc_error:
            print(f"MFCC extraction timed out or failed for {file_id}: {mfcc_error}")
            mfcc_features, mfcc_vis_path, mfcc_data_path = None, None, None
        else:
            mfcc_features, mfcc_vis_path, mfcc_data_path = mfcc_result
        
        # Check if at least one extraction was successful
        if stft_data_path or mfcc_data_path:
            feature_info = {
                'stft_path': stft_data_path,
                'stft_vis_path': stft_vis_path,
                'mfcc_path': mfcc_data_path,
                'mfcc_vis_path': mfcc_vis_path
            }
            print(f"Successfully processed {file_id}")
            return feature_info
        else:
            print(f"Feature extraction failed for {file_id}")
            return None
            
    except TimeoutError:
        print(f"ERROR: Processing timed out for {file_id}")
        return None
    except Exception as e:
        print(f"ERROR: Unexpected error processing {file_id}: {e}")
        return None

def extract_features_from_csv():
    """Extract features with improved error handling and timeouts."""
    # Ensure directories exist
    ensure_dirs_exist()
    
    # Check if CSV exists
    if not os.path.exists(CSV_PATH):
        print(f"ERROR: CSV file not found: {CSV_PATH}")
        return None
    
    # Load metadata
    try:
        df = pd.read_csv(CSV_PATH)
        print(f"Loaded CSV with {len(df)} entries")
    except Exception as e:
        print(f"ERROR: Failed to load CSV: {e}")
        return None
    
    # Add feature columns if they don't exist
    for col in ['stft_path', 'stft_vis_path', 'mfcc_path', 'mfcc_vis_path', 'features_extracted']:
        if col not in df.columns:
            df[col] = None if col != 'features_extracted' else False
    
    # Find files that need processing - look for preprocessed files
    if 'preprocessed' in df.columns and 'preprocessed_path' in df.columns:
        # Use preprocessed files if available
        to_process = df[(df['preprocessed'] == True) & 
                         (df['features_extracted'] != True) & 
                         (df['preprocessed_path'].notna())]
        file_path_col = 'preprocessed_path'
        print(f"Found {len(to_process)} preprocessed files that need feature extraction")
    else:
        # Otherwise use original files
        to_process = df[df['features_extracted'] != True]
        file_path_col = 'path'
        print(f"No preprocessed files found, using {len(to_process)} original files")
    
    if len(to_process) == 0:
        print("No files to process! Either they're all processed or none are preprocessed.")
        return df
    
    # Track success and failure
    successful = 0
    failed = 0
    
    # Process files individually - batching changed to sequential to better identify issues
    for i, (idx, row) in enumerate(tqdm(to_process.iterrows(), total=len(to_process), desc="Processing files")):
        print(f"\nProcessing file {i+1}/{len(to_process)}")
        
        file_path = row[file_path_col]
        
        # Check if file exists
        if not isinstance(file_path, str) or not os.path.exists(file_path):
            print(f"File not found or invalid path: {file_path}")
            failed += 1
            continue
        
        # Process the file with overall timeout
        try:
            # Use the timeout mechanism
            start_time = time.time()
            feature_info = process_file(file_path)
            end_time = time.time()
            
            if feature_info:
                # Update dataframe with feature paths
                for key, value in feature_info.items():
                    df.loc[idx, key] = value
                
                df.loc[idx, 'features_extracted'] = True
                successful += 1
                print(f"File processed successfully in {end_time - start_time:.2f} seconds")
            else:
                failed += 1
                print(f"File processing failed after {end_time - start_time:.2f} seconds")
            
            # Save progress after each file
            if (i + 1) % 5 == 0:
                df.to_csv(CSV_PATH, index=False)
                print(f"Progress saved. Processed {i+1}/{len(to_process)} files")
                
        except Exception as e:
            print(f"ERROR: Failed to process file {file_path}: {e}")
            failed += 1
    
    # Final save
    df.to_csv(CSV_PATH, index=False)
    
    print(f"\nFeature extraction complete. Success: {successful}, Failed: {failed}")
    return df

# Run the extraction
if __name__ == "__main__":
    print(f"PyTorch version: {torch.__version__}")
    print(f"MPS available: {torch.backends.mps.is_available()}")
    print(f"MPS built: {torch.backends.mps.is_built()}")
    
    start_time = time.time()
    print(f"Starting feature extraction from {CSV_PATH}")
    
    df = extract_features_from_csv()
    
    if df is not None:
        end_time = time.time()
        print(f"\nFeature extraction completed in {end_time - start_time:.2f} seconds!")
        print("\nNote: Run visualization script separately to create feature visualizations")
    else:
        print("\nFeature extraction failed. Check the errors above.")

Using MPS (Apple Metal) device for acceleration
PyTorch version: 2.6.0
MPS available: True
MPS built: True
Starting feature extraction from /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/audio_metadata.csv
Loaded CSV with 213 entries
Found 17 preprocessed files that need feature extraction


Processing files:   0%|          | 0/17 [00:00<?, ?it/s]


Processing file 1/17
Starting to process: preprocessed_6400504G
Loaded file: preprocessed_6400504G, Sample rate: 48000, Shape: (7661689,), Size: 14.61 MB
Processing preprocessed_6400504G using GPU
STFT extraction timed out or failed for preprocessed_6400504G: Processing timed out after 15 seconds


Processing files:   6%|▌         | 1/17 [00:30<08:01, 30.12s/it]

MFCC extraction timed out or failed for preprocessed_6400504G: Processing timed out after 15 seconds
Feature extraction failed for preprocessed_6400504G
File processing failed after 30.11 seconds

Processing file 2/17
Starting to process: preprocessed_9400807L
Loaded file: preprocessed_9400807L, Sample rate: 48000, Shape: (71356,), Size: 0.14 MB
Processing preprocessed_9400807L using GPU
STFT extraction timed out or failed for preprocessed_9400807L: Processing timed out after 15 seconds


...

Processing files:  94%|█████████▍| 16/17 [08:37<00:32, 32.70s/it]

MFCC extraction timed out or failed for preprocessed_77006009: Processing timed out after 15 seconds
Feature extraction failed for preprocessed_77006009
File processing failed after 30.04 seconds

Processing file 17/17
Starting to process: preprocessed_79040002
Loaded file: preprocessed_79040002, Sample rate: 48000, Shape: (2856960,), Size: 5.45 MB
Processing preprocessed_79040002 using GPU
STFT extraction timed out or failed for preprocessed_79040002: Processing timed out after 15 seconds


Processing files: 100%|██████████| 17/17 [09:36<00:00, 33.90s/it]

MFCC extraction timed out or failed for preprocessed_79040002: Processing timed out after 15 seconds
Feature extraction failed for preprocessed_79040002
File processing failed after 58.23 seconds

Feature extraction complete. Success: 0, Failed: 17



Feature extraction completed in 576.56 seconds!

Note: Run visualization script separately to create feature visualizations


## 4. plots

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import stft
import time
from tqdm import tqdm

# Define paths with your specific CSV path
CSV_PATH = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/audio_metadata.csv'
FEATURES_DIR = os.path.join(os.path.dirname(CSV_PATH), 'Features')
MAX_DURATION = 30  # Maximum seconds to process from any file

def create_downsampled_features(file_path, output_prefix, max_duration=30):
    """Extract features from a limited portion of audio to prevent timeouts"""
    try:
        # Load audio file
        sr, audio = wavfile.read(file_path)
        
        # Convert to mono if stereo
        if len(audio.shape) > 1 and audio.shape[1] > 1:
            audio = np.mean(audio, axis=1)
            
        # Normalize
        if audio.dtype == np.int16:
            audio = audio.astype(np.float32) / 32767.0
        elif audio.dtype == np.int32:
            audio = audio.astype(np.float32) / 2147483647.0
            
        # Limit duration to prevent timeouts
        max_samples = min(len(audio), sr * max_duration)
        audio = audio[:max_samples]
        
        # Extract STFT with modest parameters
        n_fft = min(1024, len(audio)//8)
        hop_length = n_fft // 4
        f, t, Zxx = stft(audio, fs=sr, nperseg=n_fft, noverlap=hop_length)
        
        # Convert to dB scale, capped at a reasonable range
        S_mag = np.abs(Zxx)
        S_db = 20 * np.log10(np.maximum(S_mag, 1e-10))
        
        # Downsample the spectrogram for faster visualization
        if S_db.shape[1] > 200:
            downsampling_factor = S_db.shape[1] // 200
            S_db_downsampled = S_db[:, ::downsampling_factor]
        else:
            S_db_downsampled = S_db
            
        # Save smaller spectrograms
        stft_vis_path = f"{output_prefix}_spectrogram.png"
        stft_data_path = f"{output_prefix}_stft.npy"
        
        # Create spectrogram visualization with lower resolution
        plt.figure(figsize=(8, 5), dpi=80)
        plt.pcolormesh(t, f, S_db, shading='gouraud', cmap='viridis')
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [s]')
        plt.title('Spectrogram')
        plt.colorbar(label='Magnitude [dB]')
        plt.tight_layout()
        plt.savefig(stft_vis_path)
        plt.close()
        
        # Save a downsampled version of the data
        np.save(stft_data_path, S_db_downsampled)
        
        # Create a simple MFCC-like feature (just the lower part of STFT)
        n_mfcc = min(40, S_db.shape[0]//2)
        mfcc_like = S_db[:n_mfcc]
        
        mfcc_vis_path = f"{output_prefix}_mfcc.png"
        mfcc_data_path = f"{output_prefix}_mfcc.npy"
        
        plt.figure(figsize=(8, 4), dpi=80)
        plt.pcolormesh(np.arange(mfcc_like.shape[1]), np.arange(mfcc_like.shape[0]), 
                      mfcc_like, shading='gouraud')
        plt.ylabel('MFCC Coefficient')
        plt.xlabel('Time Frame')
        plt.title('MFCC-like Features')
        plt.colorbar(label='Magnitude [dB]')
        plt.tight_layout()
        plt.savefig(mfcc_vis_path)
        plt.close()
        
        # Save downsampled MFCC data
        np.save(mfcc_data_path, mfcc_like[:, ::max(1, mfcc_like.shape[1]//100)])
        
        feature_info = {
            'stft_path': stft_data_path,
            'stft_vis_path': stft_vis_path,
            'mfcc_path': mfcc_data_path,
            'mfcc_vis_path': mfcc_vis_path
        }
        return feature_info
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def process_batch(df, features_dir, csv_path, batch_size=5, timeout_per_file=60):
    """Process files in small batches with timeouts"""
    # Create feature directories
    os.makedirs(os.path.join(features_dir, 'Spectrograms'), exist_ok=True)
    os.makedirs(os.path.join(features_dir, 'STFT'), exist_ok=True)
    os.makedirs(os.path.join(features_dir, 'MFCC_Plots'), exist_ok=True)
    os.makedirs(os.path.join(features_dir, 'MFCC'), exist_ok=True)
    
    # Find files that need processing
    to_process = df[(df['preprocessed'] == True) & (df['features_extracted'] != True)]
    print(f"Found {len(to_process)} files to process")
    
    if len(to_process) == 0:
        return df
        
    # Process in small batches
    for i in range(0, len(to_process), batch_size):
        batch = to_process.iloc[i:i+batch_size]
        print(f"Processing batch {i//batch_size + 1}/{(len(to_process)-1)//batch_size + 1}")
        
        for idx, row in tqdm(batch.iterrows(), total=len(batch)):
            file_path = row['preprocessed_path'] if 'preprocessed_path' in row and pd.notna(row['preprocessed_path']) else row['path']
            
            # Skip if file doesn't exist
            if not os.path.exists(file_path):
                print(f"File not found: {file_path}")
                continue
                
            # Extract file ID
            file_id = os.path.splitext(os.path.basename(file_path))[0]
            output_prefix = os.path.join(features_dir, file_id)
            
            # Process with timeout
            start_time = time.time()
            try:
                feature_info = create_downsampled_features(
                    file_path, 
                    output_prefix,
                    max_duration=MAX_DURATION
                )
                
                if feature_info:
                    # Update dataframe
                    for key, value in feature_info.items():
                        df.loc[idx, key] = value
                    df.loc[idx, 'features_extracted'] = True
                    print(f"Processed {file_id} in {time.time() - start_time:.2f}s")
                
                # Don't let any single file take too long
                if time.time() - start_time > timeout_per_file:
                    print(f"Processing timed out for {file_id}")
                    break
                    
            except Exception as e:
                print(f"Error processing {file_id}: {e}")
        
        # Save progress after each batch
        df.to_csv(csv_path, index=False)
        print(f"Progress saved after batch {i//batch_size + 1}")
    
    return df

def create_simple_visualizations(df, features_dir):
    """Create simple visualizations from extracted features"""
    # Create output directory
    vis_dir = os.path.join(features_dir, 'Summary')
    os.makedirs(vis_dir, exist_ok=True)
    
    # Filter to files with extracted features
    features_df = df[df['features_extracted'] == True].copy()
    if len(features_df) == 0:
        print("No files with extracted features found.")
        return
    
    # Count by type
    if 'is_biological' in features_df.columns:
        bio_count = len(features_df[features_df['is_biological'] == True])
        nonbio_count = len(features_df[features_df['is_biological'] == False])
        print(f"Biological sounds: {bio_count}")
        print(f"Non-biological sounds: {nonbio_count}")
    
    # Create a simple grid of spectrograms
    sample_files = features_df.sample(min(9, len(features_df)))
    
    plt.figure(figsize=(12, 8))
    plt.suptitle("Sample Spectrograms", fontsize=16)
    
    for i, (_, row) in enumerate(sample_files.iterrows()):
        if pd.notna(row['stft_vis_path']) and os.path.exists(row['stft_vis_path']):
            plt.subplot(3, 3, i + 1)
            img = plt.imread(row['stft_vis_path'])
            plt.imshow(img)
            
            # Create title based on available metadata
            title = row['filename'] if 'filename' in row else f"Sample {i+1}"
            if 'species_type' in row and pd.notna(row['species_type']):
                title = f"{row['species_type']}"
            
            plt.title(title, fontsize=9)
            plt.axis('off')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(os.path.join(vis_dir, 'sample_spectrograms.png'), dpi=100)
    plt.close()
    
    print(f"Created simple visualizations in {vis_dir}")
    return True

# Main execution
if __name__ == "__main__":
    try:
        print(f"Using CSV: {CSV_PATH}")
        print(f"Using Features directory: {FEATURES_DIR}")
        
        # Create features directory if it doesn't exist
        os.makedirs(FEATURES_DIR, exist_ok=True)
        
        # Load metadata CSV
        df = pd.read_csv(CSV_PATH)
        print(f"Loaded {len(df)} entries from CSV")
        
        # Add feature columns if they don't exist
        for col in ['stft_path', 'stft_vis_path', 'mfcc_path', 'mfcc_vis_path', 'features_extracted']:
            if col not in df.columns:
                df[col] = None if col != 'features_extracted' else False
        
        # Process files
        df = process_batch(df, FEATURES_DIR, CSV_PATH, batch_size=5, timeout_per_file=60)
        
        # Create simple visualizations
        create_simple_visualizations(df, FEATURES_DIR)
        
        print("Processing complete!")
        
    except Exception as e:
        print(f"Error: {e}")

Using CSV: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/audio_metadata.csv
Using Features directory: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/Features
Loaded 213 entries from CSV
Found 17 files to process
Processing batch 1/4


 20%|██        | 1/5 [00:04<00:17,  4.27s/it]

Processed preprocessed_6400504G in 4.26s


 40%|████      | 2/5 [00:04<00:06,  2.06s/it]

Processed preprocessed_9400807L in 0.51s


...

 50%|█████     | 1/2 [00:00<00:00,  2.20it/s]

Processed preprocessed_77006009 in 0.45s


100%|██████████| 2/2 [00:04<00:00,  2.25s/it]

Processed preprocessed_79040002 in 4.04s
Progress saved after batch 4
Biological sounds: 0
Non-biological sounds: 213


Created simple visualizations in /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/Features/Summary
Processing complete!


## NMF

In [3]:
import os
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF
from scipy.io import wavfile
import soundfile as sf
from tqdm import tqdm

# Define file paths
METADATA_PATH = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/audio_metadata.csv'
DATASET_PATH = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample'
OUTPUT_DIR = os.path.join(DATASET_PATH, 'NMF_Output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Parameters for NMF
N_COMPONENTS = 16  # Number of basis vectors
MAX_ITER = 200     # Maximum iterations for NMF
RANDOM_STATE = 42  # For reproducibility

def load_metadata():
    """Load metadata from CSV file."""
    print("Loading metadata...")
    df = pd.read_csv(METADATA_PATH)
    print(f"Loaded {len(df)} entries from metadata.")
    return df

def apply_nmf_to_audio(audio_path, n_components=N_COMPONENTS, sample_rate=None):
    """Apply NMF to a single audio file."""
    print(f"Processing: {audio_path}")
    
    # Load audio file
    if sample_rate is None:
        y, sr = librosa.load(audio_path, sr=None)
    else:
        y, sr = librosa.load(audio_path, sr=sample_rate)
    
    # Compute spectrogram
    S = np.abs(librosa.stft(y))
    
    # Apply NMF with the correct solver for KL divergence
    nmf = NMF(n_components=n_components, max_iter=MAX_ITER, 
              random_state=RANDOM_STATE, beta_loss='kullback-leibler',
              solver='mu')  # 'mu' solver is required for kullback-leibler
    W = nmf.fit_transform(S)  # Basis matrix
    H = nmf.components_      # Activation matrix
    
    return y, sr, S, W, H

def reconstruct_sources(W, H, n_sources=2):
    """Reconstruct sources from NMF components."""
    # For demonstration purposes, we'll split components equally between sources
    # In a real-world scenario, you'd want a more sophisticated clustering
    components_per_source = len(H) // n_sources
    
    sources = []
    for i in range(n_sources):
        start_idx = i * components_per_source
        end_idx = (i + 1) * components_per_source if i < n_sources - 1 else len(H)
        
        # Zero out components not belonging to this source
        H_source = H.copy()
        mask = np.ones_like(H_source, dtype=bool)
        mask[start_idx:end_idx, :] = False
        H_source[mask] = 0
        
        # Reconstruct spectrogram for this source
        S_source = W @ H_source
        sources.append(S_source)
    
    return sources

def spectrogram_to_audio(S, original_phase, sr, hop_length=512):
    """Convert spectrogram back to audio using original phase."""
    # Combine magnitude with phase information
    S_complex = S * np.exp(1j * original_phase)
    
    # Inverse STFT
    y = librosa.istft(S_complex, hop_length=hop_length)
    
    return y

def compute_si_sdr(estimated_source, reference_source):
    """Compute Scale-Invariant Signal-to-Distortion Ratio (SI-SDR)."""
    # Ensure same length
    min_len = min(len(estimated_source), len(reference_source))
    estimated_source = estimated_source[:min_len]
    reference_source = reference_source[:min_len]
    
    # Normalize to zero mean
    estimated_source = estimated_source - np.mean(estimated_source)
    reference_source = reference_source - np.mean(reference_source)
    
    # Calculate SI-SDR
    reference_energy = np.sum(reference_source ** 2)
    if reference_energy < 1e-10:
        return float('-inf')
    
    optimal_scaling = np.dot(estimated_source, reference_source) / reference_energy
    projection = optimal_scaling * reference_source
    
    noise = estimated_source - projection
    
    ratio = np.sum(projection ** 2) / (np.sum(noise ** 2) + 1e-10)
    sdr = 10 * np.log10(ratio + 1e-10)
    
    return sdr

def process_file_with_nmf(file_path, output_dir, n_components=N_COMPONENTS, n_sources=2):
    """Process a single file with NMF and save results."""
    # Apply NMF to audio
    y, sr, S, W, H = apply_nmf_to_audio(file_path, n_components)
    
    # Get original phase information
    phase = np.angle(librosa.stft(y))
    
    # Reconstruct sources
    source_specs = reconstruct_sources(W, H, n_sources)
    
    # Convert spectrograms back to audio using original phase
    source_audios = [spectrogram_to_audio(S_source, phase, sr) for S_source in source_specs]
    
    # Save results
    file_name = os.path.basename(file_path).replace('.wav', '')
    
    # Create visualizations directory
    vis_dir = os.path.join(output_dir, 'visualizations')
    os.makedirs(vis_dir, exist_ok=True)
    
    # Plot and save spectrograms
    plt.figure(figsize=(15, 10))
    
    plt.subplot(2, 2, 1)
    librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), 
                             y_axis='log', x_axis='time', sr=sr)
    plt.title('Original Spectrogram')
    plt.colorbar(format='%+2.0f dB')
    
    for i, S_source in enumerate(source_specs):
        plt.subplot(2, 2, i+2)
        librosa.display.specshow(librosa.amplitude_to_db(S_source, ref=np.max), 
                                 y_axis='log', x_axis='time', sr=sr)
        plt.title(f'Source {i+1} Spectrogram')
        plt.colorbar(format='%+2.0f dB')
    
    plt.tight_layout()
    plt.savefig(os.path.join(vis_dir, f"{file_name}_spectrograms.png"))
    plt.close()
    
    # Save separated audio files
    for i, source_audio in enumerate(source_audios):
        output_path = os.path.join(output_dir, f"{file_name}_source{i+1}.wav")
        
        # Normalize audio to avoid clipping
        source_audio = source_audio / (np.max(np.abs(source_audio)) + 1e-10)
        
        # Save as WAV file
        sf.write(output_path, source_audio, sr)
        print(f"Saved source {i+1} to {output_path}")
    
    return source_audios, sr

def main():
    """Main function to run NMF-based source separation on the dataset."""
    # Load metadata
    df = load_metadata()
    
    # Create output directory
    nmf_output_dir = OUTPUT_DIR
    os.makedirs(nmf_output_dir, exist_ok=True)
    
    # Filter preprocessed files
    if 'preprocessed_path' in df.columns and df['preprocessed_path'].notna().any():
        processed_files = df[df['preprocessed'] == True]['preprocessed_path'].dropna().tolist()
        print(f"Found {len(processed_files)} preprocessed files.")
    else:
        processed_files = df['path'].dropna().tolist()
        print(f"Using {len(processed_files)} original files.")
    
    # Limit number of files for testing
    max_files = 213
    processed_files = processed_files[:max_files]
    print(f"Processing {len(processed_files)} files...")
    
    # Process files
    results = []
    for file_path in tqdm(processed_files):
        if os.path.exists(file_path):
            try:
                source_audios, sr = process_file_with_nmf(file_path, nmf_output_dir)
                results.append({
                    'file_path': file_path,
                    'sources': len(source_audios),
                    'sampling_rate': sr,
                    'status': 'success'
                })
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
                results.append({
                    'file_path': file_path,
                    'status': f'error: {str(e)}'
                })
        else:
            print(f"File not found: {file_path}")
            results.append({
                'file_path': file_path,
                'status': 'file not found'
            })
    
    # Save results summary
    results_df = pd.DataFrame(results)
    results_df.to_csv(os.path.join(nmf_output_dir, 'nmf_results.csv'), index=False)
    print(f"Results saved to {os.path.join(nmf_output_dir, 'nmf_results.csv')}")
    
    print("NMF source separation completed!")

if __name__ == "__main__":
    main()

Loading metadata...
Loaded 213 entries from metadata.
Found 213 preprocessed files.
Processing 213 files...


  0%|          | 0/213 [00:00<?, ?it/s]

Processing: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6102101A.wav


  0%|          | 1/213 [00:00<02:29,  1.41it/s]

Saved source 1 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_6102101A_source1.wav
Saved source 2 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_6102101A_source2.wav
Processing: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_6102101P.wav


...

Saved source 1 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_ice-cracking2_source1.wav
Saved source 2 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_ice-cracking2_source2.wav
Processing: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_ice-cracking.wav


/opt/anaconda3/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
100%|█████████▉| 212/213 [07:28<00:02,  2.03s/it]

Saved source 1 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_ice-cracking_source1.wav
Saved source 2 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_ice-cracking_source2.wav
Processing: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/non-bio/preprocessed_Earthquake sound3.wav


/opt/anaconda3/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(
100%|██████████| 213/213 [07:40<00:00,  2.16s/it]

Saved source 1 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_Earthquake sound3_source1.wav
Saved source 2 to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/preprocessed_Earthquake sound3_source2.wav
Results saved to /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/nmf_results.csv
NMF source separation completed!


## BioCPNN net

In [4]:
"""
Helper script to fix any issues with running BioCPPNet
Copy this code to a new file in the same directory as your main script
"""

import os
import torch
import torch.nn as nn
import numpy as np
import librosa
import soundfile as sf
import pandas as pd
from tqdm import tqdm

# Check environment
print("Python environment check:")
print(f"- PyTorch version: {torch.__version__}")
print(f"- MPS available: {torch.backends.mps.is_available() if hasattr(torch.backends, 'mps') else 'Not supported in this PyTorch version'}")
print(f"- CUDA available: {torch.cuda.is_available()}")

# Check directory structure
base_dir = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample'
dirs_to_check = [
    '',
    'Sample - bio', 
    'Sample - Non Bio',
    'Preprocessed',
    'NMF_Output',
    'BioCPPNet_Output'
]

print("\nChecking directory structure:")
for d in dirs_to_check:
    path = os.path.join(base_dir, d)
    exists = os.path.exists(path)
    print(f"- {path}: {'EXISTS' if exists else 'MISSING'}")
    
    if exists and os.path.isdir(path):
        # Count files
        file_count = sum(1 for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)))
        wav_count = sum(1 for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.lower().endswith('.wav'))
        print(f"  * Contains {file_count} files ({wav_count} WAV files)")

# Check metadata file
metadata_path = os.path.join(base_dir, 'Preprocessed/audio_metadata.csv')
if os.path.exists(metadata_path):
    try:
        df = pd.read_csv(metadata_path)
        print(f"\nMetadata file: {len(df)} entries")
        print(f"Columns: {', '.join(df.columns)}")
    except Exception as e:
        print(f"\nError reading metadata file: {e}")
else:
    print(f"\nMetadata file not found: {metadata_path}")

# Simple WAV loader test
print("\nTesting audio loading:")
audio_dirs = [
    os.path.join(base_dir, 'Sample - bio'),
    os.path.join(base_dir, 'Sample - Non Bio')
]

for audio_dir in audio_dirs:
    if os.path.exists(audio_dir) and os.path.isdir(audio_dir):
        wav_files = [f for f in os.listdir(audio_dir) if f.lower().endswith('.wav')]
        
        if wav_files:
            sample_file = os.path.join(audio_dir, wav_files[0])
            try:
                y, sr = librosa.load(sample_file, sr=None)
                print(f"Successfully loaded {sample_file}")
                print(f"- Duration: {len(y)/sr:.2f} seconds")
                print(f"- Sample rate: {sr} Hz")
                print(f"- Shape: {y.shape}")
            except Exception as e:
                print(f"Error loading {sample_file}: {e}")
        else:
            print(f"No WAV files found in {audio_dir}")

print("\nEnvironment check complete!")

Python environment check:
- PyTorch version: 2.6.0
- MPS available: True
- CUDA available: False

Checking directory structure:
- /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/: EXISTS
  * Contains 1 files (0 WAV files)
- /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - bio: EXISTS
  * Contains 1 files (0 WAV files)
- /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio: EXISTS
  * Contains 1 files (0 WAV files)
- /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed: EXISTS
  * Contains 2 files (0 WAV files)
- /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output: EXISTS
  * Contains 428 files (426 WAV files)
- /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/BioCPPNet_Output: EXISTS
  * Contains 1 files (0 WAV

In [10]:
"""
Improved script to update audio metadata CSV with better non-bio file detection
"""

import os
import pandas as pd
import librosa
from tqdm import tqdm

# Paths
BASE_DIR = '/Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample'
METADATA_PATH = os.path.join(BASE_DIR, 'Preprocessed/audio_metadata.csv')
OUTPUT_METADATA_PATH = os.path.join(BASE_DIR, 'Preprocessed/updated_audio_metadata.csv')

# Additional non-bio directories to search
ADDITIONAL_NONBIO_DIRS = [
    os.path.join(BASE_DIR, 'Sample - Non Bio'),
    os.path.join(BASE_DIR, 'NMF_Output'),
]

# Function to determine if a file is biological based on its path or name
def is_biological(file_path):
    # Check if file is in a typical bio directory
    if '/bio/' in file_path.lower() or 'sample - bio' in file_path.lower():
        return True
    
    # Keywords that indicate non-biological sources
    non_bio_keywords = [
        'earthquake', 'hurricane', 'ice', 'submarine', 'ocean-wave', 
        'lightning', 'hydrothermal', 'volcano', 'storm', 'waves', 'non-bio',
        'nonbio', 'non bio'
    ]
    
    # Check for non-bio indicators in the path
    for keyword in non_bio_keywords:
        if keyword.lower() in file_path.lower():
            return False
    
    # Check specific directories that are known to be non-bio
    non_bio_dirs = [
        '/non-bio/', 
        'sample - non bio',
        '/preprocessed/non-bio/'
    ]
    
    for dir_path in non_bio_dirs:
        if dir_path.lower() in file_path.lower():
            return False
    
    # Default to biological if no non-bio indicators found
    # This is arguable, but since most files are bio, it's a reasonable default
    return True

# Function to find all WAV files in a directory recursively
def find_all_wav_files(directory):
    wav_files = []
    if os.path.exists(directory) and os.path.isdir(directory):
        for root, _, files in os.walk(directory):
            for file in files:
                if file.lower().endswith('.wav'):
                    wav_files.append(os.path.join(root, file))
    return wav_files

def update_metadata():
    """Update the metadata CSV with correct file paths and classifications"""
    print(f"Reading metadata from: {METADATA_PATH}")
    
    # Start a new DataFrame for the updated metadata
    all_files = []
    
    try:
        # Read the metadata CSV if it exists
        original_df = pd.read_csv(METADATA_PATH)
        print(f"Found {len(original_df)} entries in original metadata")
    except Exception as e:
        print(f"Error reading original metadata, starting fresh: {e}")
        original_df = pd.DataFrame()
    
    # Find all WAV files in the dataset
    print("Searching for all WAV files in the dataset...")
    
    # Search the Preprocessed directories
    bio_dir = os.path.join(BASE_DIR, 'Preprocessed/bio')
    nonbio_dir = os.path.join(BASE_DIR, 'Preprocessed/non-bio')
    
    # Get all WAV files from the main bio and non-bio directories
    all_bio_files = find_all_wav_files(bio_dir)
    all_nonbio_files = find_all_wav_files(nonbio_dir)
    
    print(f"Found {len(all_bio_files)} bio files in Preprocessed/bio")
    print(f"Found {len(all_nonbio_files)} non-bio files in Preprocessed/non-bio")
    
    # Search additional non-bio directories
    for dir_path in ADDITIONAL_NONBIO_DIRS:
        additional_files = find_all_wav_files(dir_path)
        print(f"Found {len(additional_files)} additional files in {dir_path}")
        
        # Add to appropriate list based on classification
        for file_path in additional_files:
            if not is_biological(file_path):
                all_nonbio_files.append(file_path)
            else:
                all_bio_files.append(file_path)
    
    # Additional search for NMF source files (which might be missclassified)
    nmf_output_dir = os.path.join(BASE_DIR, 'NMF_Output')
    if os.path.exists(nmf_output_dir):
        nmf_files = find_all_wav_files(nmf_output_dir)
        for file_path in nmf_files:
            # Classify source1 as bio and source2 as non-bio
            if 'source1' in file_path.lower():
                all_bio_files.append(file_path)
            elif 'source2' in file_path.lower():
                all_nonbio_files.append(file_path)
    
    # Remove duplicates
    all_bio_files = list(set(all_bio_files))
    all_nonbio_files = list(set(all_nonbio_files))
    
    print(f"After all searches: {len(all_bio_files)} bio files and {len(all_nonbio_files)} non-bio files")
    
    # Create entries for all files
    for file_path in all_bio_files:
        filename = os.path.basename(file_path)
        all_files.append({
            'filename': filename,
            'path': file_path,
            'is_biological': True,
            'updated_path': file_path,
            'updated_preprocessed_path': file_path if 'preprocessed' in file_path.lower() else None,
            'updated_is_biological': True
        })
    
    for file_path in all_nonbio_files:
        filename = os.path.basename(file_path)
        all_files.append({
            'filename': filename,
            'path': file_path,
            'is_biological': False,
            'updated_path': file_path,
            'updated_preprocessed_path': file_path if 'preprocessed' in file_path.lower() else None,
            'updated_is_biological': False
        })
    
    # Create the updated DataFrame
    updated_df = pd.DataFrame(all_files)
    
    # Keep any columns from the original DataFrame that are not in the updated one
    if not original_df.empty:
        # Get columns from original that aren't in updated
        original_cols = set(original_df.columns) - set(updated_df.columns)
        
        # For each original file, try to find the matching updated file
        for _, orig_row in original_df.iterrows():
            if 'filename' in orig_row and orig_row['filename'] in updated_df['filename'].values:
                # Find the corresponding row in updated_df
                updated_idx = updated_df[updated_df['filename'] == orig_row['filename']].index[0]
                
                # Copy over any additional columns
                for col in original_cols:
                    if col in orig_row:
                        updated_df.loc[updated_idx, col] = orig_row[col]
    
    # Save the updated metadata
    updated_df.to_csv(OUTPUT_METADATA_PATH, index=False)
    print(f"Saved updated metadata with {len(updated_df)} entries to: {OUTPUT_METADATA_PATH}")
    
    # Print some statistics
    bio_count = updated_df[updated_df['updated_is_biological'] == True].shape[0]
    nonbio_count = updated_df[updated_df['updated_is_biological'] == False].shape[0]
    print(f"Updated metadata contains {bio_count} bio files and {nonbio_count} non-bio files")
    
    # Print some examples
    print("\nExample bio files:")
    for path in updated_df[updated_df['updated_is_biological'] == True]['updated_path'].head(3):
        print(f"  {path}")
    
    print("\nExample non-bio files:")
    for path in updated_df[updated_df['updated_is_biological'] == False]['updated_path'].head(3):
        print(f"  {path}")
    
    return updated_df

if __name__ == "__main__":
    update_metadata()

Reading metadata from: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/audio_metadata.csv
Found 213 entries in original metadata
Searching for all WAV files in the dataset...
Found 210 bio files in Preprocessed/bio
Found 502 non-bio files in Preprocessed/non-bio
Found 3 additional files in /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio
Found 426 additional files in /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output
After all searches: 633 bio files and 721 non-bio files
Saved updated metadata with 1354 entries to: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Preprocessed/updated_audio_metadata.csv
Updated metadata contains 633 bio files and 721 non-bio files

Example bio files:
  /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/NMF_Output/prepr

In [13]:
"""
Modified BioCPPNet implementation that uses directory structure instead of CSV
"""

import os
import numpy as np
import librosa
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import soundfile as sf
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from itertools import permutations

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# For M1 Mac, use MPS if available
if hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS acceleration on M1 Mac: {device}")

# Constants
SAMPLE_RATE = 48000  # Based on preprocessing documentation
SEQUENCE_LENGTH = 65536  # ~1.36 seconds at 48kHz
BATCH_SIZE = 16
NUM_EPOCHS = 100
LEARNING_RATE = 0.001
N_SOURCES = 2  # Bio and non-bio

class DownsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=15, stride=1, padding=7):
        super(DownsampleBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.LeakyReLU(0.2)
        self.pool = nn.MaxPool1d(2)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return self.pool(x), x  # Return pooled output and skip connection

class UpsampleBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=15, stride=1, padding=7):
        super(UpsampleBlock, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        
    def forward(self, x, skip):
        x = self.upsample(x)
        # Ensure x and skip have the same length
        if x.size(-1) != skip.size(-1):
            x = F.pad(x, (0, skip.size(-1) - x.size(-1)))
        x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        x = self.bn(x)
        return self.relu(x)

class BioCPPNet(nn.Module):
    def __init__(self, n_sources=N_SOURCES):
        super(BioCPPNet, self).__init__()
        
        # Encoder (downsampling path)
        self.down1 = DownsampleBlock(1, 16)        # Input: Raw audio waveform
        self.down2 = DownsampleBlock(16, 32)      
        self.down3 = DownsampleBlock(32, 64)       
        self.down4 = DownsampleBlock(64, 128)      
        self.down5 = DownsampleBlock(128, 256)     
        
        # Bottleneck
        self.bottleneck = nn.Sequential(
            nn.Conv1d(256, 512, 15, 1, 7),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Conv1d(512, 512, 15, 1, 7),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2)
        )
        
        # Decoder (upsampling path)
        self.up1 = UpsampleBlock(512 + 256, 256)
        self.up2 = UpsampleBlock(256 + 128, 128)
        self.up3 = UpsampleBlock(128 + 64, 64)
        self.up4 = UpsampleBlock(64 + 32, 32)
        self.up5 = UpsampleBlock(32 + 16, 16)
        
        # Output layer - multiple sources
        self.out_conv = nn.Conv1d(16, n_sources, 1)
        
    def forward(self, x):
        # Encoder path with skip connections
        x1, skip1 = self.down1(x)
        x2, skip2 = self.down2(x1)
        x3, skip3 = self.down3(x2)
        x4, skip4 = self.down4(x3)
        x5, skip5 = self.down5(x4)
        
        # Bottleneck
        x = self.bottleneck(x5)
        
        # Decoder path with skip connections
        x = self.up1(x, skip5)
        x = self.up2(x, skip4)
        x = self.up3(x, skip3)
        x = self.up4(x, skip2)
        x = self.up5(x, skip1)
        
        # Output layer
        x = self.out_conv(x)
        
        return x

class DirectoryAudioDataset(Dataset):
    def __init__(self, bio_dir, nonbio_dir, sequence_length=SEQUENCE_LENGTH, sr=SAMPLE_RATE):
        self.sequence_length = sequence_length
        self.sr = sr
        
        # Initialize file lists
        self.bio_files = []
        self.nonbio_files = []
        
        print("Creating dataset from directory structure...")
        
        # Scan bio directory recursively
        if os.path.exists(bio_dir):
            print(f"Scanning bio directory: {bio_dir}")
            for root, dirs, files in os.walk(bio_dir):
                for file in files:
                    if file.endswith('.wav'):
                        file_path = os.path.join(root, file)
                        self.bio_files.append(file_path)
            print(f"Found {len(self.bio_files)} bio audio files")
        else:
            print(f"Warning: Bio directory not found: {bio_dir}")
        
        # Scan non-bio directory recursively
        if os.path.exists(nonbio_dir):
            print(f"Scanning non-bio directory: {nonbio_dir}")
            for root, dirs, files in os.walk(nonbio_dir):
                for file in files:
                    if file.endswith('.wav'):
                        file_path = os.path.join(root, file)
                        self.nonbio_files.append(file_path)
            print(f"Found {len(self.nonbio_files)} non-bio audio files")
        else:
            print(f"Warning: Non-bio directory not found: {nonbio_dir}")
            
            # Fallback: if no dedicated non-bio directory, try looking for a separate "Preprocessed/non-bio" folder
            fallback_nonbio_dir = os.path.join(os.path.dirname(bio_dir), "Preprocessed/non-bio")
            if os.path.exists(fallback_nonbio_dir):
                print(f"Using fallback non-bio directory: {fallback_nonbio_dir}")
                for root, dirs, files in os.walk(fallback_nonbio_dir):
                    for file in files:
                        if file.endswith('.wav'):
                            file_path = os.path.join(root, file)
                            self.nonbio_files.append(file_path)
                print(f"Found {len(self.nonbio_files)} non-bio audio files from fallback directory")
        
        # Print a few paths for verification
        if self.bio_files:
            print("\nExample bio file paths:")
            for path in self.bio_files[:3]:
                print(f"  {path}")
        
        if self.nonbio_files:
            print("\nExample non-bio file paths:")
            for path in self.nonbio_files[:3]:
                print(f"  {path}")
                
        # If still no non-bio files, use environmental sounds from preprocessed directory
        if not self.nonbio_files:
            print("No non-bio files found. Attempting to find environmental sound files...")
            base_dir = os.path.dirname(os.path.dirname(bio_dir))
            
            # Common environment sound keywords to look for
            env_sound_keywords = ['ocean', 'wave', 'storm', 'wind', 'rain', 'thunder', 
                                 'earthquake', 'volcano', 'ice', 'submarine', 'ship', 
                                 'engine', 'hydrothermal', 'lightning']
            
            for root, dirs, files in os.walk(base_dir):
                for file in files:
                    if file.endswith('.wav'):
                        # Check if the file name contains environmental keywords
                        # and is not already in bio_files
                        file_path = os.path.join(root, file)
                        file_lower = file.lower()
                        
                        if any(keyword in file_lower for keyword in env_sound_keywords) and file_path not in self.bio_files:
                            self.nonbio_files.append(file_path)
            
            print(f"Found {len(self.nonbio_files)} environmental sound files")
            
            if self.nonbio_files:
                print("\nExample environmental sound file paths:")
                for path in self.nonbio_files[:3]:
                    print(f"  {path}")
        
        # Ensure we have files to work with
        if len(self.bio_files) == 0:
            raise ValueError("No bio files found. Cannot proceed with training.")
        
        if len(self.nonbio_files) == 0:
            raise ValueError("No non-bio files found. Cannot proceed with training.")

    def __len__(self):
        # Use the minimum of bio and non-bio files
        return min(len(self.bio_files), len(self.nonbio_files)) * 10  # Generate 10 examples per pair
    
    def __getitem__(self, idx):
        # Randomly select one bio and one non-bio file
        bio_idx = np.random.randint(0, len(self.bio_files))
        nonbio_idx = np.random.randint(0, len(self.nonbio_files))
        
        # Load bio audio file
        try:
            bio_audio, _ = librosa.load(self.bio_files[bio_idx], sr=self.sr, mono=True)
        except Exception as e:
            print(f"Error loading bio file {self.bio_files[bio_idx]}: {e}")
            # Fall back to another bio file
            fallback_idx = (bio_idx + 1) % len(self.bio_files)
            bio_audio, _ = librosa.load(self.bio_files[fallback_idx], sr=self.sr, mono=True)
        
        # Load non-bio audio file
        try:
            nonbio_audio, _ = librosa.load(self.nonbio_files[nonbio_idx], sr=self.sr, mono=True)
        except Exception as e:
            print(f"Error loading non-bio file {self.nonbio_files[nonbio_idx]}: {e}")
            # Fall back to another non-bio file
            fallback_idx = (nonbio_idx + 1) % len(self.nonbio_files)
            nonbio_audio, _ = librosa.load(self.nonbio_files[fallback_idx], sr=self.sr, mono=True)
        
        # Make sure they're the right length
        if len(bio_audio) < self.sequence_length:
            bio_audio = np.pad(bio_audio, (0, self.sequence_length - len(bio_audio)))
        else:
            start = np.random.randint(0, len(bio_audio) - self.sequence_length)
            bio_audio = bio_audio[start:start + self.sequence_length]
            
        if len(nonbio_audio) < self.sequence_length:
            nonbio_audio = np.pad(nonbio_audio, (0, self.sequence_length - len(nonbio_audio)))
        else:
            start = np.random.randint(0, len(nonbio_audio) - self.sequence_length)
            nonbio_audio = nonbio_audio[start:start + self.sequence_length]
        
        # Normalize
        bio_audio = bio_audio / (np.max(np.abs(bio_audio)) + 1e-10)
        nonbio_audio = nonbio_audio / (np.max(np.abs(nonbio_audio)) + 1e-10)
        
        # Create the mixture (can adjust mixing ratios if needed)
        mix_ratio = np.random.uniform(0.3, 0.7)  # Random mix between sources
        mixture = mix_ratio * bio_audio + (1 - mix_ratio) * nonbio_audio
        
        # Normalize mixture
        mixture = mixture / (np.max(np.abs(mixture)) + 1e-10)
        
        # Convert to tensors
        mixture_tensor = torch.tensor(mixture.reshape(1, -1), dtype=torch.float32)
        sources_tensor = torch.tensor(np.stack([bio_audio, nonbio_audio]), dtype=torch.float32)
        
        return mixture_tensor, sources_tensor

class PITLoss(nn.Module):
    """Permutation Invariant Training Loss for source separation"""
    def __init__(self):
        super(PITLoss, self).__init__()
        self.mse_loss = nn.MSELoss()
    
    def forward(self, est_sources, true_sources):
        # est_sources: [batch, n_sources, time]
        # true_sources: [batch, n_sources, time]
        
        batch_size = est_sources.shape[0]
        n_sources = est_sources.shape[1]
        
        # Compute all possible permutations
        perms = torch.tensor(list(permutations(range(n_sources))), dtype=torch.long)
        
        min_loss = torch.zeros(batch_size, device=est_sources.device).fill_(float('inf'))
        
        for p in perms:
            # Permute the estimated sources
            perm_sources = est_sources[:, p, :]
            
            # Calculate MSE loss for this permutation
            loss = torch.mean((perm_sources - true_sources) ** 2, dim=(1, 2))
            
            # Update min_loss if this permutation is better
            min_loss = torch.minimum(min_loss, loss)
        
        return torch.mean(min_loss)

def train_model(model, train_loader, valid_loader, num_epochs=NUM_EPOCHS, lr=LEARNING_RATE):
    """Train the BioCPPNet model"""
    criterion = PITLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)
    
    best_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch_idx, (mixtures, sources) in enumerate(progress_bar):
            mixtures = mixtures.to(device)
            sources = sources.to(device)
            
            # Forward pass
            optimizer.zero_grad()
            outputs = model(mixtures)
            
            # Adjust dimensions to match sources shape
            if outputs.shape[-1] != sources.shape[-1]:
                outputs = F.pad(outputs, (0, sources.shape[-1] - outputs.shape[-1]))
            
            # Calculate loss
            loss = criterion(outputs, sources)
            
            # Backward and optimize
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            progress_bar.set_postfix({'loss': train_loss / (batch_idx + 1)})
        
        train_loss /= len(train_loader)
        
        # Validation phase
        model.eval()
        valid_loss = 0.0
        
        with torch.no_grad():
            for mixtures, sources in valid_loader:
                mixtures = mixtures.to(device)
                sources = sources.to(device)
                
                outputs = model(mixtures)
                
                # Adjust dimensions if needed
                if outputs.shape[-1] != sources.shape[-1]:
                    outputs = F.pad(outputs, (0, sources.shape[-1] - outputs.shape[-1]))
                
                loss = criterion(outputs, sources)
                valid_loss += loss.item()
        
        valid_loss /= len(valid_loader)
        
        # Update learning rate
        scheduler.step(valid_loss)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Valid Loss: {valid_loss:.4f}")
        
        # Save best model
        if valid_loss < best_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), "best_biocppnet_model.pt")
            print(f"Model saved with loss {best_loss:.4f}")
    
    return model

def compute_si_sdr_batch(estimated_sources, reference_sources):
    """Compute Scale-Invariant SDR between batches of estimated and reference sources"""
    # estimated_sources: [n_sources, time]
    # reference_sources: [n_sources, time]
    
    # Ensure sources are on CPU for numpy operations
    estimated_sources = estimated_sources.cpu().numpy()
    reference_sources = reference_sources.cpu().numpy()
    
    n_sources = estimated_sources.shape[0]
    si_sdr_sum = 0
    
    for i in range(n_sources):
        # Get single source estimates
        est_source = estimated_sources[i]
        ref_source = reference_sources[i]
        
        # Normalize to zero mean
        est_source = est_source - np.mean(est_source)
        ref_source = ref_source - np.mean(ref_source)
        
        # Calculate SI-SDR
        reference_energy = np.sum(ref_source ** 2)
        if reference_energy < 1e-10:
            si_sdr = 0  # Avoid division by zero
        else:
            optimal_scaling = np.dot(est_source, ref_source) / reference_energy
            projection = optimal_scaling * ref_source
            noise = est_source - projection
            
            ratio = np.sum(projection ** 2) / (np.sum(noise ** 2) + 1e-10)
            si_sdr = 10 * np.log10(ratio + 1e-10)
        
        si_sdr_sum += si_sdr
    
    return si_sdr_sum / n_sources

def evaluate_model(model, test_loader):
    """Evaluate the trained model using SI-SDR"""
    model.eval()
    test_si_sdr = 0.0
    
    with torch.no_grad():
        for mixtures, sources in tqdm(test_loader, desc="Evaluating"):
            mixtures = mixtures.to(device)
            sources = sources.to(device)
            
            # Forward pass
            estimated_sources = model(mixtures)
            
            # Adjust dimensions if needed
            if estimated_sources.shape[-1] != sources.shape[-1]:
                estimated_sources = F.pad(estimated_sources, (0, sources.shape[-1] - estimated_sources.shape[-1]))
            
            # Calculate SI-SDR for each sample in the batch
            for i in range(mixtures.shape[0]):
                si_sdr_sample = compute_si_sdr_batch(estimated_sources[i], sources[i])
                test_si_sdr += si_sdr_sample
    
    test_si_sdr /= len(test_loader.dataset)
    print(f"Test SI-SDR: {test_si_sdr:.4f} dB")
    return test_si_sdr

def separate_audio(model, audio_path, output_dir):
    """Separate a single audio file into bio and non-bio components"""
    # Load audio file
    y, sr = librosa.load(audio_path, sr=SAMPLE_RATE, mono=True)
    
    # Process in chunks of SEQUENCE_LENGTH with overlap
    chunk_size = SEQUENCE_LENGTH
    hop_size = chunk_size // 2  # 50% overlap
    
    # Initialize output arrays
    bio_output = np.zeros_like(y)
    nonbio_output = np.zeros_like(y)
    
    # Process each chunk
    model.eval()
    with torch.no_grad():
        for i in range(0, len(y) - chunk_size + 1, hop_size):
            # Extract chunk
            chunk = y[i:i+chunk_size]
            
            # Convert to tensor
            chunk_tensor = torch.tensor(chunk.reshape(1, 1, -1), dtype=torch.float32).to(device)
            
            # Separate sources
            separated_sources = model(chunk_tensor)
            
            # Convert back to numpy
            separated_chunk = separated_sources.squeeze().cpu().numpy()
            
            # Apply window for smooth blending (Hann window)
            window = np.hanning(chunk_size)
            
            # Add to output arrays with windowing
            bio_output[i:i+chunk_size] += separated_chunk[0] * window
            nonbio_output[i:i+chunk_size] += separated_chunk[1] * window
    
    # Normalize outputs
    bio_output = bio_output / np.max(np.abs(bio_output) + 1e-10)
    nonbio_output = nonbio_output / np.max(np.abs(nonbio_output) + 1e-10)
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Save separated audio files
    file_name = os.path.basename(audio_path).replace('.wav', '')
    bio_path = os.path.join(output_dir, f"{file_name}_bio.wav")
    nonbio_path = os.path.join(output_dir, f"{file_name}_nonbio.wav")
    
    sf.write(bio_path, bio_output, sr)
    sf.write(nonbio_path, nonbio_output, sr)
    
    return bio_path, nonbio_path

def find_nonbio_files(base_dir):
    """Find non-bio audio files from typical folder names or file patterns"""
    nonbio_files = []
    
    # Common folders for non-biological sounds
    nonbio_folders = ['non-bio', 'nonbio', 'environment', 'environmental', 
                     'noise', 'ambient', 'ocean', 'wave', 'earthquake', 
                     'Sample - Non Bio']
    
    # Common non-bio sound keywords
    nonbio_keywords = ['ocean', 'wave', 'storm', 'wind', 'rain', 'thunder', 
                      'earthquake', 'volcano', 'ice', 'submarine', 'ship', 
                      'engine', 'hydrothermal', 'lightning']
    
    # Search for non-bio files
    for root, dirs, files in os.walk(base_dir):
        folder_name = os.path.basename(root)
        
        # Check if this is a non-bio folder
        is_nonbio_folder = any(nonbio_name in folder_name.lower() for nonbio_name in nonbio_folders)
        
        for file in files:
            if file.endswith('.wav'):
                file_path = os.path.join(root, file)
                
                # Add file if it's in a non-bio folder
                if is_nonbio_folder:
                    nonbio_files.append(file_path)
                    continue
                
                # Or if the filename contains non-bio keywords
                if any(keyword in file.lower() for keyword in nonbio_keywords):
                    nonbio_files.append(file_path)
    
    return nonbio_files

def main():
    """Main function for training and evaluating BioCPPNet using directory structure"""
    # Configure paths - adjust these based on your environment
    # Use the structure shown in the PDF
    base_dir = os.path.join(os.path.expanduser('~'), 'Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample')
    bio_dir = os.path.join(base_dir, 'Sample - bio')
    nonbio_dir = os.path.join(base_dir, 'Sample - Non Bio')
    output_dir = os.path.join(base_dir, 'BioCPPNet_Output')
    
    print(f"Base directory: {base_dir}")
    print(f"Bio directory: {bio_dir}")
    print(f"Non-bio directory: {nonbio_dir}")
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # If non-bio directory doesn't exist, attempt to find non-bio files elsewhere
    if not os.path.exists(nonbio_dir):
        print(f"Non-bio directory not found: {nonbio_dir}")
        nonbio_dir = None  # Will be handled by the dataset class
    
    # Create dataset directly from directories
    dataset = DirectoryAudioDataset(bio_dir, nonbio_dir)
    
    # Check if we have enough data
    if len(dataset.bio_files) == 0 or len(dataset.nonbio_files) == 0:
        print("Not enough data to train the model. Please ensure you have both bio and non-bio audio files.")
        return
    
    # Split dataset
    train_size = int(0.8 * len(dataset))
    valid_size = int(0.1 * len(dataset))
    test_size = len(dataset) - train_size - valid_size
    
    train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
        dataset, [train_size, valid_size, test_size]
    )
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # Initialize model
    model = BioCPPNet().to(device)
    print(f"Model initialized on {device}")
    
    # Train model
    model = train_model(model, train_loader, valid_loader)
    
    # Evaluate model
    si_sdr = evaluate_model(model, test_loader)
    
    # Save final model
    torch.save(model.state_dict(), os.path.join(output_dir, "biocppnet_final_model.pt"))
    
    # Process test files from NMF directory if it exists
    nmf_dir = os.path.join(base_dir, 'NMF_Output')
    if os.path.exists(nmf_dir):
        print(f"Using NMF output directory for test files: {nmf_dir}")
        # Find files that look like they might be mixed signals (not source separated)
        test_files = [
            os.path.join(nmf_dir, f) 
            for f in os.listdir(nmf_dir) 
            if f.endswith('.wav') and "source" not in f.lower()
        ]
        
        # If we didn't find any, just use the first 5 WAV files we can find
        if not test_files:
            test_files = [
                os.path.join(nmf_dir, f) 
                for f in os.listdir(nmf_dir) 
                if f.endswith('.wav')
            ][:5]
    else:
        # Fall back to preprocessed files if NMF output doesn't exist
        print("NMF output directory not found. Looking for test files in the dataset...")
        test_files = []
        
        # Choose some representative files for testing
        test_files = dataset.bio_files[:3]  # Use first 3 bio files
        if dataset.nonbio_files:
            test_files.extend(dataset.nonbio_files[:2])  # Add first 2 non-bio files
        
    print(f"Found {len(test_files)} test files for processing")
    
    if not test_files:
        print("No test files found. Cannot demonstrate separation.")
        return
    
    # Process test files
    print(f"Processing {len(test_files)} test files...")
    
    for file_path in test_files:
        try:
            bio_path, nonbio_path = separate_audio(model, file_path, output_dir)
            print(f"Processed {file_path} -> Bio: {bio_path}, Non-bio: {nonbio_path}")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")
    
    print("BioCPPNet processing completed!")

if __name__ == "__main__":
    main()  

Using device: cpu
Using MPS acceleration on M1 Mac: mps
Base directory: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample
Bio directory: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - bio
Non-bio directory: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio
Creating dataset from directory structure...
Scanning bio directory: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - bio
Found 210 bio audio files
Scanning non-bio directory: /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - Non Bio
Found 44 non-bio audio files

Example bio file paths:
  /Users/yashsawrikar/Documents/VIT Labs/Semester 8/Senior Design Project/Dataset - Sample/Sample - bio/False Killer Whale ( Pseudorca crassidens )/6102101A.wav
  /Users/yashsawrikar/Documents/VIT Labs/Semester 8

Epoch 1/100: 100%|██████████| 22/22 [26:31<00:00, 72.35s/it, loss=0.0618] 


Epoch 1/100, Train Loss: 0.0618, Valid Loss: 0.0442
Model saved with loss 0.0442


Epoch 2/100: 100%|██████████| 22/22 [31:32<00:00, 86.02s/it, loss=0.0199] 


Epoch 2/100, Train Loss: 0.0199, Valid Loss: 0.0230
Model saved with loss 0.0230


...

Epoch 17/100, Train Loss: 0.0139, Valid Loss: 0.0119
Model saved with loss 0.0119


Epoch 18/100: 100%|██████████| 22/22 [3:19:44<00:00, 544.77s/it, loss=0.0141]  


Epoch 18/100, Train Loss: 0.0141, Valid Loss: 0.0154


Epoch 19/100:  18%|█▊        | 4/22 [34:02<2:33:12, 510.68s/it, loss=0.0141]


KeyboardInterrupt: 

In [1]:
import keras
import tensorflow as tf

In [5]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim):
        self.W = self.add_weight([input_dim, output_dim])
        self.b = self.add_weight([output_dim])

    def call(self, inputs):
        z =  tf.matmul(inputs, self.W) + self.b
    
        output=tf.math.sigmoid(z)
        return output

In [6]:
layer = tf.keras.layers.Dense(units=2)